<a href="https://colab.research.google.com/github/gamecicn/TF2_Study/blob/Kaggle_HousePrice/Kaggle_HousePrice_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
# First XGBoost model for Pima Indians dataset
import os
import pandas as pd
import numpy as np

from numpy import loadtxt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [156]:
DATA_FOLDER = "/content/drive/MyDrive/DS_data/HousePrice"

TRAIN_FILE = os.path.join(DATA_FOLDER, "train.csv")
TEST_FILE  = os.path.join(DATA_FOLDER, "test.csv")

# load data
train_data_raw = pd.read_csv(TRAIN_FILE)
test_data_raw = pd.read_csv(TEST_FILE)

# Remove Column ID
train_data_raw = train_data.iloc[:, 0:]

# EDA

### Data Over view

In [159]:
print("Train data shape:", train_data_raw.shape) 
print("Test  data shape:", train_data_raw.shape) 

Train data shape: (1121, 37)
Test  data shape: (1121, 37)


In [160]:
train_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1121 entries, 0 to 1459
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1121 non-null   int64  
 1   LotFrontage    1121 non-null   float64
 2   LotArea        1121 non-null   int64  
 3   OverallQual    1121 non-null   int64  
 4   OverallCond    1121 non-null   int64  
 5   MasVnrArea     1121 non-null   float64
 6   BsmtFinSF1     1121 non-null   int64  
 7   BsmtFinSF2     1121 non-null   int64  
 8   BsmtUnfSF      1121 non-null   int64  
 9   TotalBsmtSF    1121 non-null   int64  
 10  1stFlrSF       1121 non-null   int64  
 11  2ndFlrSF       1121 non-null   int64  
 12  LowQualFinSF   1121 non-null   int64  
 13  GrLivArea      1121 non-null   int64  
 14  BsmtFullBath   1121 non-null   int64  
 15  BsmtHalfBath   1121 non-null   int64  
 16  FullBath       1121 non-null   int64  
 17  HalfBath       1121 non-null   int64  
 18  BedroomA

In [161]:
train_data.select_dtypes(include="float64").head()

,LotFrontage,MasVnrArea,GarageYrBlt
0,65.0,196.0,7.0
1,80.0,0.0,34.0
2,68.0,162.0,9.0
3,60.0,0.0,12.0
4,84.0,350.0,10.0


In [162]:
train_data_raw.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,GarageCars,GarageArea,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold,SalePrice
0,60,65.0,8450,7,5,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,0,61,0,0,0,0,0,2,2,548,7,7,7.0,2,208500
1,20,80.0,9600,6,8,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,298,0,0,0,0,0,0,5,2,460,34,34,34.0,3,181500
2,60,68.0,11250,7,5,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,0,42,0,0,0,0,0,9,2,608,9,8,9.0,2,223500
3,70,60.0,9550,7,5,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,0,35,272,0,0,0,0,2,3,642,95,40,12.0,4,140000
4,60,84.0,14260,8,5,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,192,84,0,0,0,0,0,12,3,836,10,10,10.0,2,250000


# Transformation

Transform data type

### Choose target columns

In [163]:
# Create mini databset
YEAR_COL = ["YearBuilt", "YearRemodAdd", "GarageYrBlt", "YrSold"]

NUMERIC_COL = ["MSSubClass", 
              "LotFrontage", "LotArea",
               "OverallQual", "OverallCond",
               "MasVnrArea", 
               "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF",
               "1stFlrSF","2ndFlrSF","LowQualFinSF","GrLivArea",
               "BsmtFullBath","BsmtHalfBath",
               "FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr",
               "TotRmsAbvGrd", "Fireplaces", 
               "WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch",
               "ScreenPorch","PoolArea", "MiscVal","MoSold",
               "GarageCars", "GarageArea", 
               *YEAR_COL 
               ]


OUTPUT_COL = ["SalePrice"]

TARGET_COL = [*NUMERIC_COL, *OUTPUT_COL]

train_data = train_data_raw[TARGET_COL]

### Conver Year To Duration

In [164]:
LATEST_YEAR = 2010
train_data[YEAR_COL] = LATEST_YEAR - train_data[YEAR_COL]

In [165]:
train_data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,GarageCars,GarageArea,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold,SalePrice
0,60,65.0,8450,7,5,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,0,61,0,0,0,0,0,2,2,548,2003,2003,2003.0,2008,208500
1,20,80.0,9600,6,8,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,298,0,0,0,0,0,0,5,2,460,1976,1976,1976.0,2007,181500
2,60,68.0,11250,7,5,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,0,42,0,0,0,0,0,9,2,608,2001,2002,2001.0,2008,223500
3,70,60.0,9550,7,5,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,0,35,272,0,0,0,0,2,3,642,1915,1970,1998.0,2006,140000
4,60,84.0,14260,8,5,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,192,84,0,0,0,0,0,12,3,836,2000,2000,2000.0,2008,250000


### Impute Missing data

In [166]:
train_data.dropna(inplace=True)
train_data.shape

(1121, 37)

### Split X, Y

In [167]:
# Create X, Y 
X_train = train_data.iloc[:, 0:-1]
Y_train = train_data.iloc[:, -1]
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,GarageCars,GarageArea,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
0,60,65.0,8450,7,5,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,0,61,0,0,0,0,0,2,2,548,2003,2003,2003.0,2008
1,20,80.0,9600,6,8,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,298,0,0,0,0,0,0,5,2,460,1976,1976,1976.0,2007
2,60,68.0,11250,7,5,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,0,42,0,0,0,0,0,9,2,608,2001,2002,2001.0,2008
3,70,60.0,9550,7,5,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,0,35,272,0,0,0,0,2,3,642,1915,1970,1998.0,2006
4,60,84.0,14260,8,5,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,192,84,0,0,0,0,0,12,3,836,2000,2000,2000.0,2008


### Scale

In [168]:
scaler = preprocessing.StandardScaler().fit(X_train[NUMERIC_COL])

X_train[NUMERIC_COL] = scaler.transform(X_train[NUMERIC_COL])

X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,GarageCars,GarageArea,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold
0,0.092295,-0.233570,-0.205885,0.570704,-0.525499,0.462009,0.571581,-0.282645,-0.986817,-0.496377,-0.822753,1.142290,-0.109875,0.341150,1.144540,-0.237934,0.762485,1.240077,0.175732,-0.207047,0.895154,-0.955163,-0.760257,0.231036,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,-1.615345,0.185499,0.235641,0.992930,0.823953,0.953084,0.153084
1,-0.865696,0.384834,-0.064358,-0.153825,2.284122,-0.572748,1.152559,-0.282645,-0.689078,0.415045,0.229055,-0.805853,-0.109875,-0.514644,-0.808321,4.064069,0.762485,-0.765877,0.175732,-0.207047,-0.362324,0.626429,1.686090,-0.716739,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,-0.498715,0.185499,-0.224712,0.120665,-0.460746,-0.092969,-0.596291
2,0.092295,-0.109889,0.138702,0.570704,-0.525499,0.282510,0.101672,-0.282645,-0.355789,-0.352705,-0.656951,1.169664,-0.109875,0.486330,1.144540,-0.237934,0.762485,1.240077,0.175732,-0.207047,-0.362324,0.626429,-0.760257,-0.064173,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,0.990125,0.185499,0.549518,0.928317,0.776371,0.875599,0.153084
3,0.331793,-0.439705,-0.070512,0.570704,-0.525499,-0.572748,-0.475034,-0.282645,-0.120264,-0.720866,-0.550734,0.918733,-0.109875,0.354522,1.144540,-0.237934,-1.056123,-0.765877,0.175732,-0.207047,0.266415,0.626429,-0.760257,-0.172934,4.083851,-0.11253,-0.278676,-0.072999,-0.141407,-1.615345,1.714502,0.727382,-1.850006,-0.746235,0.759371,-1.345665
4,0.092295,0.549742,0.509132,1.295234,-0.525499,1.275032,0.462647,-0.282645,-0.231360,0.152394,-0.074052,1.596249,-0.109875,1.273355,1.144540,-0.237934,0.762485,1.240077,1.471756,-0.207047,1.523893,0.626429,0.815913,0.588393,-0.356622,-0.11253,-0.278676,-0.072999,-0.141407,2.106755,1.714502,1.742250,0.896011,0.681208,0.836856,0.153084


# Build Model

In [171]:
model = XGBRegressor()
model.fit(X_train, Y_train)

[16:16:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

# Predict

In [172]:
test_data = test_data_raw[TARGET_COL[:-1]]


test_data[YEAR_COL] = LATEST_YEAR - test_data[YEAR_COL]

test_data[NUMERIC_COL] = scaler.transform(test_data[NUMERIC_COL])

y_pred = model.predict(test_data)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [173]:
y_pred

array([106301.95, 145393.72, 157473.1 , ..., 151384.8 ,  93000.34,
       194379.08], dtype=float32)

### Write to submit file

In [174]:
answer = pd.DataFrame(data={"Id" : range(1461, 1461+len(y_pred)), 
                            "SalePrice" : y_pred})

answer.head()

,Id,SalePrice
0,1461,106301.953125
1,1462,145393.718750
2,1463,157473.093750
3,1464,157545.531250
4,1465,180708.484375


In [175]:
OUTPUT_PATH = "/content/drive/MyDrive/output"
OUTPUT_FILE_NAME = "kaggle_house_price_3.csv"

answer.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_FILE_NAME), index=False)